<a href="https://colab.research.google.com/github/kyungmin0101/medical_ai/blob/main/240220/regression_youjin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import csv
from scipy.ndimage import gaussian_filter
import os

base_directory1 = '/content/drive/MyDrive/Refractive_Index_Measurements_New/1stSeries_07.05.21'

all_csv_X1 = []
all_csv_Y1 = []

for root, dirs, files in os.walk(base_directory1):
    for file in files:
        if file.endswith('.CSV') and file != 'REF.CSV':
            file_path = os.path.join(root, file)

            with open(file_path, 'rt') as f:
                reader = csv.reader(f)
                csv_content = [raw for raw in reader]
            f.close()
            df = pd.DataFrame(csv_content)[24:].reset_index(drop=True)
            df = df.astype(float)
            df.columns = ['X', 'Y']
            Y = float(os.path.basename(root))
            all_csv_X1.append(df)
            all_csv_Y1.append(Y)

base_directory2 = '/content/drive/MyDrive/Refractive_Index_Measurements_New/2ndSeries_21.05.21'

all_csv_X2 = []
all_csv_Y2 = []

for root, dirs, files in os.walk(base_directory2):
    for file in files:
        if file.endswith('.CSV') and file != 'REF.CSV':
            file_path = os.path.join(root, file)

            with open(file_path, 'rt') as f:
                reader = csv.reader(f)
                csv_content = [raw for raw in reader]
            f.close()
            df = pd.DataFrame(csv_content)[24:].reset_index(drop=True)
            df = df.astype(float)
            df.columns = ['X', 'Y']
            Y = float(os.path.basename(root))
            all_csv_X2.append(df)
            all_csv_Y2.append(Y)

In [4]:
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks

df_Y = pd.DataFrame(all_csv_Y1, columns=['Y'])

analysis_values = []
for df, label in zip(all_csv_X1, all_csv_X2):
    smoothed_signal = gaussian_filter(df['Y'], sigma=40)
    peaks, _ = find_peaks(smoothed_signal)

    peak_values = df['Y'][peaks].reset_index(drop=True)
    peak_times = df['X'][peaks].reset_index(drop=True)

    time_diff = np.diff(peak_times)
    std_signal_peak = np.std(time_diff)
    mean_signal_peak = np.mean(time_diff)

    analysis_values.append([std_signal_peak, mean_signal_peak])

data = pd.DataFrame(analysis_values, columns=['std', 'mean'])

spectrum_data = pd.concat([data, df_Y], axis=1)
print(spectrum_data)

df_Y_test = pd.DataFrame(all_csv_Y2, columns=['Y'])

analysis_values2 = []
for df, label in zip(all_csv_X2, all_csv_Y2):
    smoothed_signal = gaussian_filter(df['Y'], sigma=40)
    peaks, _ = find_peaks(smoothed_signal)

    peak_values = df['Y'][peaks].reset_index(drop=True)
    peak_times = df['X'][peaks].reset_index(drop=True)

    time_diff = np.diff(peak_times)
    std_signal_peak = np.std(time_diff)
    mean_signal_peak = np.mean(time_diff)

    analysis_values2.append([std_signal_peak, mean_signal_peak])

data = pd.DataFrame(analysis_values2, columns=['std','mean'])

spectrum_test = pd.concat([data, df_Y_test], axis=1)
print(spectrum_test)



          std      mean     Y
0    0.149360  3.232519  1.31
1    0.150904  3.229835  1.31
2    0.143360  3.230373  1.31
3    0.146847  3.229300  1.31
4    0.147209  3.229300  1.31
..        ...       ...   ...
226  0.056895  2.973859  1.42
227  0.054118  2.979688  1.42
228  0.055064  2.975024  1.42
229  0.055808  2.979694  1.42
230  0.054665  2.971524  1.42

[231 rows x 3 columns]
          std      mean     Y
0    0.158012  3.175647  1.31
1    0.143858  3.172516  1.31
2    0.145312  3.170426  1.31
3    0.143976  3.174082  1.31
4    0.164553  3.178258  1.31
..        ...       ...   ...
226  0.045806  2.718267  1.44
227  0.044999  2.714950  1.44
228  0.066097  2.689914  1.44
229  0.065139  2.692743  1.44
230  0.069293  2.692743  1.44

[231 rows x 3 columns]


In [65]:
from sklearn.model_selection import train_test_split

train, test= train_test_split(spectrum_test, test_size = 0.3, random_state=1, shuffle=True)

X_train1 = spectrum_data[['mean','std']]
y_train1= spectrum_data['Y']

X_train2 = train[['mean','std']]
y_train2= train['Y']

X_train = pd.concat([X_train1, X_train2], axis=0)
y_train = pd.concat([y_train1, y_train2], axis=0)

X_test = test[['mean','std']]
y_test = test['Y']

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(392, 2)
(392,)
(70, 2)
(70,)


In [66]:
from lightgbm import LGBMRegressor
import time
lgb = LGBMRegressor(max_depth=2, learning_rate=0.1, n_estimators=100, force_col_wise=True)
lgb.fit(X_train, y_train)

start_lgb = time.time()
y_pred = lgb.predict(X_test)
end_lgb = time.time()

from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 20, random_state = 15)
rfr.fit(X_train, y_train)

start_rfr = time.time()
y_pred_rfr = rfr.predict(X_test)
end_rfr = time.time()

print('lgb: ', end_lgb-start_lgb, 's')
print('rfr: ', end_rfr-start_rfr, 's')

[LightGBM] [Info] Total Bins 258
[LightGBM] [Info] Number of data points in the train set: 392, number of used features: 2
[LightGBM] [Info] Start training from score 1.397857
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [67]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np

MSE = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_pred, y_test)
RMSE = np.sqrt(MSE)
print('lgb MSE:', MSE)
print('lgb RMSE:', RMSE)
print('lgb R2:', r2)

MSE = mean_squared_error(y_test, y_pred_rfr)
r2 = r2_score(y_pred_rfr, y_test)
RMSE = np.sqrt(MSE)
print('rfr MSE:', MSE)
print('rfr RMSE:', RMSE)
print('rfr R2:', r2)



lgb MSE: 0.00044955673365879695
lgb RMSE: 0.021202752973583334
lgb R2: 0.8167177919576616
rfr MSE: 0.00032508214285714287
rfr RMSE: 0.01803003446633264
rfr R2: 0.8864059235413364
